In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1) Đọc dữ liệu sạch
df = pd.read_csv("../data/processed/prices_clean.csv")

# 2) Chuẩn hoá cột thời gian
df["Date"] = pd.to_datetime(df["Date"])
df["LoaiNongSan"] = df["LoaiNongSan"].astype(str).str.strip()
df = df.sort_values(["LoaiNongSan", "Date"]).reset_index(drop=True)

# 3) Chọn loại nông sản
products = df["LoaiNongSan"].unique()
print("Danh sách nông sản:", products)

product = products[0]
print("Đang phân tích:", product)

# 4) Tạo chuỗi thời gian
ts = (df[df["LoaiNongSan"].eq(product)]
      .set_index("Date")["Gia_clean"]
      .sort_index())

# 5) Đảm bảo tần suất tháng đều
ts = ts.asfreq("MS")

# 6) Nội suy nếu thiếu
if ts.isna().sum() > 0:
    ts = ts.interpolate().ffill().bfill()

print("Số điểm dữ liệu:", len(ts))
print("Thiếu sau xử lý:", int(ts.isna().sum()))
print("Khoảng thời gian:", ts.index.min().date(), "->", ts.index.max().date())

# 7) Chia train/test
h = 6
train = ts.iloc[:-h]
test = ts.iloc[-h:]

print("Train:", train.index.min().date(), "->", train.index.max().date(), "| n =", len(train))
print("Test :", test.index.min().date(),  "->", test.index.max().date(),  "| n =", len(test))

# 8) Vẽ biểu đồ
plt.figure()
plt.plot(train, label="Train")
plt.plot(test, label="Test")
plt.title(f"Train/Test split - {product}")
plt.legend()
plt.show()
